In [ ]:

from langgraph.graph import StateGraph, START, END
from typing import TypedDict, Annotated
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
class ChatState(TypedDict):

  messages : Annotated[list[BaseMessage], add_messages]

In [ ]:
llm = ChatOpenAI()

In [ ]:
def chat_node(state: ChatState):

  #take user query from state
  messages = state['messages']

  #send to llm
  response = llm.invoke(messages)

  #response store state
  return {'messages': [response]}

In [ ]:
checkpointer = MemorySaver()

graph = StateGraph(ChatState)

graph.add_node('chat_node', chat_node)

graph.add_edge(START , 'chat_node')
graph.add_edge('chat_node', END)

chatbot = graph.compile(checkpointer=checkpointer)

In [ ]:
chatbot

In [ ]:
#This will cause error now run below cell
initial_state = {
  'messages': [HumanMessage(content="what is the capital od Pakistan")]
}

chatbot.invoke(initial_state)['messages'][-1].content

In [ ]:
thread_id = '1'

while True :

  user_message = input("enter your messages")

  print('User:' , user_message)

  if user_message.strip().lower() in ['exit', 'quit' , 'bye']:
    break
  
  config = {'configurable':{'thread_id': thread_id}}
  
  response = chatbot.invoke({'messages': [HumanMessage(content = user_message)]}, config=config)

  print("AI", response['messages'][-1].content)

In [ ]:
chatbot.get_state(config=config)